In [1]:
## Here we load raw datasets and leave them in a suitable shape for the analysis.

# Datasets should be disaggregated at their maximum. They are:
# Imports, exports and wages, from Brasil and Chile.

In [30]:
import pandas as pd

## Brazil. 
### Load classification info

In [31]:
# Load info
hard_drive_path = './../../../../../media/miglesia/Elements/Matias_Iglesias_Backup/miglesia/Documents/compute-corr-structures/'
path = hard_drive_path + 'data/disagg_struct_refs/formatted/'

CIIU = pd.read_csv(path+'CIIU_full.csv')
CBO = pd.read_csv(path+'CBO_full.csv').astype(str)
CNAE = pd.read_csv(path+'CNAE_full.csv').astype(str)
HS = pd.read_csv(path+'hs_full.csv').astype(str)

#fix dtypes of IDs
for col in CBO.columns:
    if 'ID' in col:
        CBO[col] = CBO[col].astype(int).astype(str)

CNAE_agg_ref = CNAE[sorted([col for col in CNAE.columns if 'ID' in col])]
CBO_agg_ref = CBO[sorted([col for col in CBO.columns if 'ID' in col])]
HS_agg_ref = HS[sorted([col for col in HS.columns if 'ID' in col])]

exhange_rate = {'2011': 1.673, '2012': 1.953, '2013': 2.156, '2014': 2.353}
# world bank https://data.worldbank.org/indicator/PA.NUS.FCRF?end=2014&locations=BR&start=2011


### Load data. 

In [32]:
# wages_list = []

# path = hard_drive_path + 'data/econ_data/sources/'

# for y in ['2011', '2012', '2013', '2014']:
#     f = path+'rais-'+y+'-microregions-classes-families.csv'
#     wages = pd.read_csv(f)[[u'Year', u'BRA ID', u'CNAE ID', u'CBO ID', u'Total Monthly Wages',
#            u'Total Jobs', u'Total Establishments', u'Average Monthly Wage',
#            u'Average age', u'Estimated Employees']]
#     wages['Total Yearly Wages'] = 12 * wages['Total Monthly Wages'] / exhange_rate[y]
#     wages = wages.dropna(subset = ['Total Jobs', 'Total Yearly Wages'])
#     wages = wages[[u'Year', u'BRA ID', u'CNAE ID', u'CBO ID', u'Total Jobs', u'Total Yearly Wages']].rename(
#         {'BRA ID' : 'BRA ID MIR','CNAE ID':'CNAE ID CNC', 'CBO ID':'CBO ID CB4'}, axis = 1)

#     wages_list += [wages]

# wages_bra = pd.concat(wages_list)

trade_list = []
for y in ['2011', '2012', '2013', '2014']:
    f = path+'secex-'+y+'-microregions-position.csv'
    trade = pd.read_csv(f)[[u'Year', u'BRA ID', u'HS ID', u'Exports', u'Imports']].rename({'BRA ID' : 'BRA ID MIR'}, axis = 1)
    trade['HS ID'] = trade['HS ID'].astype(str).str[-4:]
    trade = trade.dropna(subset = ['Exports', 'Imports'])
    trade.rename({'HS ID': 'HS ID HS4'}, axis = 1, inplace = True) 
#     trade = trade.merge(HS_agg_labels).drop_duplicates()
    trade_list += [trade]
    
trade_bra = pd.concat(trade_list)
    

IOError: File ./../../../../../media/miglesia/Elements/Matias_Iglesias_Backup/miglesia/Documents/compute-corr-structures/data/disagg_struct_refs/formatted/secex-2011-microregions-position.csv does not exist

In [29]:
# ### Save

trade_bra.to_csv(hard_drive_path + 'data/econ_data/trade_bra.csv', index = False)
# wages_bra.to_csv(hard_drive_path + 'data/econ_data/wages_bra.csv', index = False)

## Chile. 
### Load data

In [12]:
wages_output = pd.read_csv(hard_drive_path + 'data/econ_data/sources/production_USD.csv')[['year', 'comuna_id', 'ciiu4_ori', 'output_USD', 'intermediates_USD', 'labour_cost_USD']].rename(
    {'year': 'Year', 'comuna_id':'CHI ID COM', 'ciiu4_ori': 'CIIU ID CI4', 'labour_cost_USD': 'Total Yearly Wages'}, axis = 1)

#Select years
wages_output = wages_output.loc[wages_output.Year.isin([2011, 2012, 2013, 2014])]

In [15]:
exports = pd.read_csv(hard_drive_path + 'data/econ_data/sources/exports_comuna_hs6.csv').rename({'comuna_datachile_id': 'comuna_id', 'year': 'Year'}, axis='columns')
imports = pd.read_csv(hard_drive_path + 'data/econ_data/sources/imports_comuna_hs6.csv').rename({'comuna_datachile_id': 'comuna_id', 'year': 'Year'}, axis='columns')


exports = exports.loc[exports.Year.isin([2011, 2012, 2013, 2014])]
imports = imports.loc[imports.Year.isin([2011, 2012, 2013, 2014])]

exports['pcode6_f'] = exports['pcode6_f'].astype(int).astype(str)
exports = exports.rename({'pcode6_f': 'HS ID HS6', 'comuna_id':'CHI ID COM' ,'fob_value_usd' : 'Exports'}, axis = 1)

imports['pcode6_f'] = imports['pcode6_f']#.astype(int)#.astype(str)
imports = imports.rename({'pcode6_f': 'HS ID HS6', 'comuna_id':'CHI ID COM', 'cif_value_usd' : 'Imports'}, axis = 1)

trade_chi = exports.merge(imports, how = 'outer').fillna(0)

In [16]:
# exports.to_csv('data/econ_data/exports_chi.csv', index = False)
# imports.to_csv('data/econ_data/imports_chi.csv', index = False)
trade_chi.to_csv(hard_drive_path + 'data/econ_data/trade_chi.csv', index = False)
# wages.to_csv('data/wages_chi.csv', index = False)
wages_output.to_csv(hard_drive_path + 'data/econ_data/wages_output_chi.csv', index = False)